In [1]:
from rfid_app import db
from rfid_app.models import Student, Bus, Route, TripHistory, BusStop, bus_and_routes, bus_stops_and_routes
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas as pd

/Users/john_jensen/coding/rfid_user_tracker/ENV/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
# engine = create_engine('postgresql://localhost:5432/rfid_app')
# Session = sessionmaker(bind=engine)
# session = Session() # just use db.session

In [3]:
conn_str='postgresql://localhost:5432/rfid_app'

In [4]:
pd.read_sql('SELECT students.first_name, students.last_name, students.present, \
                bus_stops.name AS bus_stop, routes.name AS route_name, buses.number AS bus_number\
            FROM bus_stops, bus_stop_route, students, routes, buses, bus_route \
            WHERE students.home_stop_id=bus_stop_route.stop_id \
            AND students.home_stop_id=bus_stops.id \
            AND bus_stop_route.route_id=routes.id \
            AND bus_route.bus_id=buses.id \
            AND bus_stop_route.route_id=bus_route.route_id'
            , con=conn_str)

,first_name,last_name,present,bus_stop,route_name,bus_number
0,bob,boblast,False,mercer,blue,45
1,sara,omaley,False,greenlake,green,321
2,steve,mcqueen,False,westlake,green,321
3,sue,test,False,queenanne,blue,45
4,tom,cat,False,caphill,orange,99


In [5]:
pd.read_sql('SELECT * \
            FROM bus_stops, bus_stop_route, bus_route, buses \
            WHERE bus_stops.id=bus_stop_route.stop_id \
            AND bus_route.route_id=bus_stop_route.stop_id \
            AND buses.id=bus_route.bus_id \
            AND buses.number=45'
            , con=conn_str)

,id,name,stop_id,route_id,route_id,bus_id,id,number
0,1,mercer,1,1,1,2,2,45


In [6]:
bus_stop = pd.read_sql('SELECT bus_stops.name AS bus_stop \
            FROM bus_stops, buses, bus_stop_route, bus_route \
            WHERE buses.number={} \
            AND bus_stops.id=bus_stop_route.stop_id \
            AND bus_stop_route.route_id=bus_route.route_id \
            AND bus_route.bus_id=buses.id'.format(45)
            , con=conn_str)

In [7]:
b =bus_stop['bus_stop']

In [8]:
b.tolist()

['mercer', 'queenanne']

In [9]:
Bus.query.filter_by(number=45).first().number

45

In [16]:
TripHistory.query.all()

[<TripHistory 1>,
 <TripHistory 2>,
 <TripHistory 4>,
 <TripHistory 5>,
 <TripHistory 3>]

In [42]:
t = session.query(TripHistory).all()
t

[<TripHistory 1>,
 <TripHistory 2>,
 <TripHistory 4>,
 <TripHistory 5>,
 <TripHistory 3>]

In [99]:
pd.read_sql('SELECT * from trip_history', db.session.bind)

,id,student_id,trip_start,trip_end,bus_stop
0,1,1,2018-04-03 16:55:48.114660,2018-04-03 16:57:55.339066,4
1,2,2,2018-04-03 17:19:07.046954,2018-04-03 17:18:54.876457,1
2,4,4,2018-04-03 17:20:36.436413,2018-04-03 17:20:24.866709,2
3,5,5,2018-04-03 17:22:37.813738,2018-04-03 17:22:24.836973,3
4,3,3,2018-04-03 17:19:38.837478,2018-04-03 17:23:34.316103,5
5,7,2,2018-04-11 17:03:06.017930,NaT,4
6,8,3,2018-04-11 17:04:11.196729,NaT,1


In [94]:
trip=TripHistory()
trip.student_id=3
trip.bus_stop=1
db.session.add(trip)
db.session.commit()

In [96]:
db.session.query(TripHistory).all()

[<TripHistory 1>,
 <TripHistory 2>,
 <TripHistory 4>,
 <TripHistory 5>,
 <TripHistory 3>,
 <TripHistory 7>,
 <TripHistory 8>]

In [103]:
t=db.session.query(TripHistory).all()

In [ ]:
db.session.query(TripHistory).filter_by

In [98]:
session